<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Bayesian Statistics with Baseball Data: Priors and Posteriors


---

To review the general setup for Bayesian modeling, we have:

$$P\left(\;model\;|\;data\;\right) = \frac{P\left(\;data\;|\;model\;\right)}{P(\;data\;)} P\left(\;model\;\right)\ .$$

This can also be written as:

$$posterior \propto likelihood \cdot prior$$

where the posterior is an update of our prior belief, given the data observed.


---

### 1. Write functions to calculate the binomial likelihood and log likelihood

The binomial likelihood is defined as:

$${\rm likelihood}(k\;|\;n,\;p) = \binom{n}{k}p^k(1-p)^{n-k}$$

Because the original can break easily with high counts, the log likelihood is often used in its place:

$$\log({\rm likelihood}) = \log\binom{n}{k}+k \cdot \log(p)+(n-k) \cdot \log(1-p)$$

To calculate factorials of large numbers, one can use the Stirling approximation to the factorial valid for large $n$:

$$
n!\approx\sqrt{2\pi n}\left(\frac{n}{e}\right)^n
$$



**Your functions should:**

1. Take a probability $p$, number of trials $n$, and number of successes $k$.
2. Return a likelihood for the trials and successes at that probability.

> **Hint:** Recall that `np.log()` can be used for natural log. `np.exp()` is useful for getting your likelihood out when the log-likelihood function is done computing. `scipy.special.comb()` can get the combinations.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from scipy.special import comb

### 2. Calculate likelihoods using both functions for:

    n=10, k=3
    n=10, k=7
    n=20, k=15
    n=50, k=9
    n=70, k=50
    n=100, k=96
    
**For probabilities:**

    p = [0.05, 0.5, 0.95]

## Review: the Beta distribution

---

[The Beta distribution](https://en.wikipedia.org/wiki/Beta_distribution) is the appropriate prior distribution for modeling Bernoulli and/or Binomial processes (occurrences of successes or failures, etc.). It is a model of the random behavior of data related to percentages, rates, proportions, etc.

The Beta distribution takes two parameters: $Beta(\alpha,\beta)$

The $\alpha$ or **alpha** parameter can be thought of as the number of `successes + 1`

The $\beta$ or **beta** parameter can be thought of as the number of `failures + 1`

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f3/Beta_distribution_pdf.svg/650px-Beta_distribution_pdf.svg.png" alt="betapdf" style="width: 400px;"/>

### 3. Plot Beta probability density functions

Use the `scipy.stats.beta` object to calculate the probability density of the beta function across a range of points.

Make one plot for each of the $n, k$ pairs you calculated the likelihood for above (converting them into success, failure pairs for the alpha, beta parameters). Plot the probability density function values across a range of probabilities between 0 and 1.

http://docs.scipy.org/doc/scipy-0.17.0/reference/generated/scipy.stats.beta.html

In [2]:
from scipy.stats import beta

## Review: the Beta as a "conjugate prior"

---

The beta distribution is a **conjugate prior** for the binomial (and the beta) distributions. What does this mean?

Take the calculation of the posterior distribution with a binomial likelihood function (any binary outcome data):

$$\text{beta posterior} \propto \text{binomial likelihood} \cdot \text{beta prior}$$

**The Beta distribution being a "conjugate prior" of the Binomial likelihood guarantees that the posterior distribution will also be a Beta distribution.**

> **Note:** Conjugate priors are extremely useful for calculating posteriors directly. Unfortunately, in many modeling scenarios we do not have the convenience of a conjugate prior. This is where methods like Markov Chain Monte Carlo (MCMC) will come into play down the line.

## Bayesian analysis of batting averages

---

Load in the simple batting avergage data for players below. There are just four fields in the dataset: the player's name, their times at bat, their hits, and their batting average.

> **Note:** [This section of the lab is a partial replication of this exercise in R, if you're interested without the "empirical Bayes" part.](https://www.r-bloggers.com/understanding-empirical-bayes-estimation-using-baseball-statistics/) 

In [3]:
hits = pd.read_csv('../../../../resource-datasets/baseball_hits/career_hits.csv')

### 4. Find the top and bottom 3 hitters according to their average.

What is wrong with using the average to find the 3 best and 3 worst hitters?

### 5. Set up a prior distribution to represent our belief about batting average

[After doing a quick search online](https://www.google.com/search?q=average+batting+average+for+players+mlb&oq=average+batting+average+for+players+mlb&aqs=chrome..69i57j0.7373j0j4&sourceid=chrome&ie=UTF-8), it looks like the overall batting average for MLB baseball players is around 0.260.

**Let's make it simple and say that our prior belief is: out of 100 at-bats we have seen 26 hits. Set up a beta distribution with `alpha=27` and `beta=75`. Plot it with the function from above.**

This is our distribution of beliefs on the batting average (probability of hit while at-bat) for MLB players.

### 6. Calculate the Maximum A Posteriori (MAP) estimate of players' batting averages

The Maximum A Posteriori (MAP) estimate is the mode of a posterior distribution for a statistical parameter.

In our case, the MAP estimate for our players' batting averages will be the mode of the posterior Beta distribution we get from updating our prior distribution with their at-bats and hits.

---

> **Note:** For an additional overview of the maths behind updating the Beta distribution posterior and conjugate priors [see this website](https://alexanderetz.com/2015/07/25/understanding-bayes-updating-priors-via-the-likelihood/).

We need to update our beta distribution prior belief about batting averages with a player's at-bat and hit information to give us a _new_ beta posterior distribution for that player's batting average. 

Luckily, with the beta distribution the update is just a matter of adding in our new observations to the alpha and beta parameters of the distribution, where alpha is the number of hits and beta is the number of misses/strikes:

    observed_hits = n_hits
    observed_misses = n_misses
    beta_prior = Beta(prior_hits+1, prior_misses+1)
    beta_posterior = Beta(prior_hits + n_hits + 1, prior_misses + n_misses + 1)
    
> **Note:** This process will also be useful in the context of A/B testing, which we will look at later on.

For each player, update the prior to the posterior distribution and calculate the mode of the distribution. The mode of a beta distribution is conveniently calculated:

$$ \frac{\alpha - 1}{\alpha + \beta -2} $$

which means we don't even really need to use scipy's beta distribution function at all. Just calculate the new alpha (hits) and beta (misses) for each player's posterior beta distribution and plug them into the formula above to get the MAP estimate of batting average.

### 7. Look up the top and bottom batters according to the MAP estimate

### 8. Plot the MAP batting average against the MLE batting average